In [1]:
%load_ext autoreload
%autoreload 2

import meshio
import numpy as np
import torch
import trimesh

from fem import fea, eval
from graphs import build_graph
from utils import msh_to_trimesh, visualize

In [ ]:
def make_dataset(
    mesh: meshio.Mesh,
    f: np.ndarray,
    points: np.ndarray,
    stl="cantilever.stl",
    msh="cantilever.msh",
):
    """Generate n graphs from given contact points on one given mesh."""
    graphs = []
    for p, f_vec in zip(points, f):
        contacts = [(p, f_vec)]

        domain, stresses_vm = fea(
            contacts,
            contact_radius=2.0,
            debug=False,
            filename_stl=stl,
            filename_msh=msh,
        )
        y = eval(domain, stresses_vm, mesh.points)
        graphs.append(build_graph(mesh, y, contacts))

    return graphs


In [8]:
stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)

points = np.array(
    [
        [0.0, 5.0, 10.0],
        [0.0, 5.0, 20.0],
        [0.0, 5.0, 30.0],
        [0.0, 5.0, 40.0],
    ]
)
f = np.tile([10.0, 0.0, 0.0], (len(points), 1))

graphs = make_dataset(mesh, f, points, stl=stl, msh=msh)
torch.save(graphs, "cantilever_4.pt")


Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'
Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'
Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'
Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'


In [7]:
stl = "meshes/cantilever.stl"
msh = "meshes/cantilever.msh"
mesh = meshio.read(msh)


f = np.array([[10.0, 0.0, 0.0]])
points = np.array([[0.0, 5.0, 30.0]])
contacts = list(zip(points, f))

domain, stresses_vm = fea(
    contacts, contact_radius=2.0, debug=False, filename_stl=stl, filename_msh=msh
)
y = eval(domain, stresses_vm, mesh.points)
graph = build_graph(mesh, y, contacts)
print("Graph is valid?", graph.validate(raise_on_error=True))


Info    : Reading 'meshes/cantilever.msh'...
Info    : 27 entities
Info    : 4923 nodes
Info    : 26950 elements
Info    : Done reading 'meshes/cantilever.msh'
Graph is valid? True


In [3]:
mesh = meshio.read(msh)
mesh = msh_to_trimesh(mesh)
visualize(mesh, graph, jupyter_backend="html", force_arrows=True)

2025-12-26 00:09:26.004 (  10.289s) [    772DF1E5F740]vtkXOpenGLRenderWindow.:1458  WARN| bad X server connection. DISPLAY=


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [ ]:
idx = np.where(np.linalg.norm(graph.x[:, 3:6], axis=1) != 0)
graph.x[idx]

In [ ]:
points = trimesh.points.PointCloud(mesh.vertices, colors=[255, 0, 0, 255])
scene = trimesh.Scene([points])
scene.show()


In [ ]:
mesh = trimesh.load_mesh(stl)
points, face_id = trimesh.sample.sample_surface(mesh, count=10)
forces = np.random.uniform(low=-10.0, high=10.0, size=(10, 3))
forces

array([[-2.08438759, -9.73864869, -9.97389612],
       [-3.8489987 ,  8.38943479, -2.99371513],
       [-5.63144316,  3.73658766, -6.61674055],
       [-6.38848162,  2.65656988,  4.35499333],
       [ 4.38787664,  9.90995507,  2.84762621],
       [-2.48682508, -4.20780071,  8.8820575 ],
       [-5.01309592, -8.87322867,  2.50777115],
       [-3.93947128, -2.33881271, -9.64329815],
       [ 2.02287619,  1.00443358, -1.05150085],
       [ 6.72801239,  9.87786886,  5.86769675]])

In [ ]:
import meshio

mesh = meshio.read("meshes/cantilever.msh")
tri_cells = [c.data for c in mesh.cells if c.type == "triangle"]
faces = np.vstack(tri_cells)
print("============================")
print("Mesh:", type(mesh))
print("Number of vertices:", mesh.points.shape[0])
print("Number of faces:", faces.shape[0])
# mesh = trimesh.Trimesh(vertices=mesh.points, faces=faces, process=False)
# mesh.show()